In [1]:
import cv2
import time
import json
import keras
import tensorflow
import numpy as np
from PIL import Image
import mediapipe as mp
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [2]:
file_name = "directory_reverse_lookup.json"

# Load the JSON data from the file into a dictionary
with open(file_name, "r") as json_file:
    directory_reverse_lookup = json.load(json_file)


print(directory_reverse_lookup)

{'0': 'A', '1': 'B', '2': 'C', '3': 'D', '4': 'del', '5': 'E', '6': 'F', '7': 'G', '8': 'H', '9': 'I', '10': 'J', '11': 'K', '12': 'L', '13': 'M', '14': 'N', '15': 'nothing', '16': 'O', '17': 'P', '18': 'Q', '19': 'R', '20': 'S', '21': 'space', '22': 'T', '23': 'U', '24': 'V', '25': 'W', '26': 'X', '27': 'Y', '28': 'Z'}


In [3]:
num_classes = len(directory_reverse_lookup.keys())


model = Sequential()

model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(50, 50, 3)))

model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation="relu"))

model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation="relu"))

model.add(Dropout(0.5))

model.add(Dense(num_classes, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Load the saved model weights into the new model
model.load_weights("model_weights.h5")

In [4]:
camera = cv2.VideoCapture(0)
def capture_image():
    return_value,frame = camera.read()

    return frame


In [5]:
mphands = mp.solutions.hands
hands = mphands.Hands(max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils
cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


_, frame = cap.read()

h, w, c = frame.shape

# print(h,w)

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(framergb)
    hand_landmarks = result.multi_hand_landmarks
    if hand_landmarks:
        for handLMs in hand_landmarks:
            x_max = 0
            y_max = 0
            x_min = w
            y_min = h
            for lm in handLMs.landmark:
                x, y = int(lm.x * w), int(lm.y * h)
                if x > x_max:
                    x_max = x
                if x < x_min:
                    x_min = x
                if y > y_max:
                    y_max = y
                if y < y_min:
                    y_min = y
            cv2.rectangle(
                frame,
                (x_min - 15, y_min - 15),
                (x_max + 15, y_max + 15),
                (0, 255, 0),
                2,
            )
            label = cv2.rectangle(
                frame,
                (x_min - 15, y_min - 15),
                (x_max + 15, y_max + 15),
                (0, 255, 0),
                2,
            )
            # cv2.rectangle(frame, (70, 70), (w-70,h-70), (0, 0,255), 2)

            start = time.time()

            img_crop = frame[y_min - 15 : y_max + 15, x_min - 15 : x_max + 15]
            img_crop = Image.fromarray(np.uint8(img_crop))
            img_crop = img_crop.resize((50, 50))
            img_crop = np.array(img_crop)
            img_crop = np.array(img_crop, dtype="float32")
            img_crop = img_crop / 255
            img_crop = img_crop.reshape((1, 50, 50, 3))

            test_datagen = ImageDataGenerator()
            test_generator = test_datagen.flow(img_crop, shuffle=False)

            predict_test = model.predict(test_generator)
            predicted_label = np.argmax(predict_test)

            end = time.time()

            cv2.putText(
                label,
                str(directory_reverse_lookup[str(predicted_label)]),
                (x - 60, y - 60),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.9,
                (36, 255, 12),
                2,
            )
            # label= cv2.flip(label,1)
            # print(predicted_label)

            fps = 1 / (end - start)

            # string formatting for fps
            cv2.putText(
                frame,
                f"{fps:.2f} Label",
                (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.9,
                (36, 255, 12),
                2,
            )

    cv2.imshow("Alphabet", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 27ms/step
